In [58]:
import os
import csv
import glob
import MySQLdb
import scrapy
from selenium import webdriver
from scrapy.selector import Selector
from scrapy.http import Request
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from sqlalchemy import *
import numpy as np
import mysql.connector
import lxml
from lxml import etree as ElementTree
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import ast
from urllib.parse import urljoin

In [59]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"
driver_search=webdriver.Chrome(desired_capabilities=caps, executable_path='C://Users/USER/Desktop/sougou/chromedriver')
driver=webdriver.Chrome(desired_capabilities=caps, executable_path='C://Users/USER/Desktop/sougou/chromedriver')
driver_search.get("https://www.wsj.com/search/term.html?min-date=2014/04/18&max-date=2018/04/18&page=1&isAdvanced=true&daysback=4y&andor=AND&sort=date-desc&source=wsjarticle,wsjblogs,wsjvideo,interactivemedia,sitesearch")
driver.get("https://www.wsj.com/articles/in-a-u-s-china-tech-battle-lots-of-self-inflicted-wounds-1524143966")

In [60]:
df=pd.DataFrame(columns=['author', 'comment', 'bio','twitter','email'])
while len(df)<2000:
    sleep(1)
    count=[]
    bio=[]
    name=[]
    twitter=[]
    mail=[]
    sel = Selector(text=driver_search.page_source)
    listings=sel.xpath('//h3[@class="headline"]//@href').extract()
    while len(listings)==0:
        sleep(1)
        sel = Selector(text=driver_search.page_source)
        listings=sel.xpath('//h3[@class="headline"]//@href').extract()
    for listing in listings:
        driver.get(urljoin('https://www.wsj.com',listing))
        sleep(1)
        article=Selector(text=driver.page_source)
        comment=article.xpath('//a[@class="comments_header"]/text()').extract()
        if len(comment)==0:
            sleep(2)
            article=Selector(text=driver.page_source)
            comment=article.xpath('//a[@class="comments_header"]/text()').extract()
            if len(comment)==0:
                pass
            else:
                countlist=comment[1]
                count=int(''.join(x for x in countlist if x.isdigit()))
        else:
            countlist=comment[1]
            count=int(''.join(x for x in countlist if x.isdigit()))
        authors=article.xpath('//div[@class="byline"]//@data-scrim').extract()
        for author in authors:
            info=ast.literal_eval(author)
            name=info['header']
            index=info['list']
            for things in index:
                if things['icon']=='bio':
                    bio=things['url']
                if things['icon']=='twitter':
                    twitter=things['url']
                if things['icon']=='email':
                    mail=things['url']
            df.loc[len(df)]=[name,count,bio,twitter,mail]
    next_page = sel.xpath('//li[@class="next-page"]/a/@href').extract_first()
    driver_search.get(urljoin('https://www.wsj.com/search/term.html',next_page))

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.16299 x86_64)


In [148]:
data=pd.read_csv('wsjcount.csv')

In [164]:
df1=df.drop_duplicates('author')

In [166]:
result=pd.merge(data,df1,how='inner',on='author')

In [167]:
result.to_csv('result.csv')

In [138]:
authordata[['comment']]=authordata[['comment']].apply(pd.to_numeric)

In [140]:
commentavg=authordata.groupby(['author']).mean()

In [105]:
commentavg.to_csv('commentavg.csv')

In [129]:
for i in range(len(df)):
    df['author'][i]=df['author'][i].replace(' and ','')

In [146]:
count.to_csv('wsjcount.csv')

In [145]:
commentavg

,comment
author,
A.J. Baime,35.500000
Aaron Back,1.166667
Abigail Shrier,264.000000
Abu Bakr Bashir,97.000000
Adam Kirsch,5.000000
Adria Calatayud,0.000000
Ahmed Deeb for The Wall Street Journal,59.000000
Aisha Al-Muslim,2.200000
Akane Otani,0.200000
